### open csv files from directory and create markdown tables

- get csv files from the local build input/image/source folder
- print out markdown files
- save files as input/includes files
- !check out how I made excel files...

**IF EXCEL IS SOURCE USE CONVERT TO CSV**

In [35]:
from pathlib import Path
import pandas as pd
import csv
from json import loads
from IPython.display import Markdown, display

In [36]:
# get path from IG output dir
# in_path = Path(r'/Users/ehaas/Documents/FHIR/davinci-ecdx/input/images/data-element-mapping.csv')
# in_path = Path(r'/Users/ehaas/Documents/FHIR/davinci-ecdx/input/includes/ballot-intro.csv')
in_path= Path(r'/Users/ehaas/Documents/FHIR/davinci-ecdx/input/data/cross-reference.csv')
out_path = Path(r'/Users/ehaas/Documents/Python/Jupyter/MyNotebooks/utils/out')
in_path.name

'cross-reference.csv'

### load CSV using pandas

trim all strings too


In [37]:
df = pd.read_csv(in_path)
df.fillna('-', inplace=True)
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
df

,IG Section,IG Section Title,IG Section Link,RI Section Title,RI Section Link,TestScript Item,TestScript Item Title,TestScript Item Link,Comment
0,3,Direct Query,https://hl7.org/fhir/us/davinci-cdex/direct-qu...,-,-,-,-,-,-
1,3.2,Benefits,https://hl7.org/fhir/us/davinci-cdex/direct-qu...,-,-,-,-,-,-
2,3.3,Sequence Diagram,https://hl7.org/fhir/us/davinci-cdex/direct-qu...,-,-,-,-,-,-
3,3.4,Discovery of Patient FHIR IDs,https://hl7.org/fhir/us/davinci-cdex/direct-qu...,-,-,-,-,-,-
4,3.5,Direct Query Transaction Scenarios,https://hl7.org/fhir/us/davinci-cdex/direct-qu...,-,-,-,-,-,-
5,3.5.1,Scenario 1,https://hl7.org/fhir/us/davinci-cdex/direct-qu...,-,-,-,-,-,-
6,3.5.2,Scenario 2,https://hl7.org/fhir/us/davinci-cdex/direct-qu...,-,-,-,-,-,-
7,3.5.3,Scenario 3,https://hl7.org/fhir/us/davinci-cdex/direct-qu...,-,-,-,-,-,-
8,3.6,Provenance,https://hl7.org/fhir/us/davinci-cdex/direct-qu...,-,-,-,-,-,-
9,3.6.1,Example of Direct Query Response Including Pro...,https://hl7.org/fhir/us/davinci-cdex/direct-qu...,-,-,-,-,-,-


In [38]:
def combine_columns(row):
    return f"[{row['IG Section']} {row['IG Section Title']}]({row['IG Section Link']})"

# Apply the custom function to create a new column 'Combined'
df['IG link'] = df.apply(combine_columns, axis=1)
df

,IG Section,IG Section Title,IG Section Link,RI Section Title,RI Section Link,TestScript Item,TestScript Item Title,TestScript Item Link,Comment,IG link
0,3,Direct Query,https://hl7.org/fhir/us/davinci-cdex/direct-qu...,-,-,-,-,-,-,[3 Direct Query](https://hl7.org/fhir/us/davin...
1,3.2,Benefits,https://hl7.org/fhir/us/davinci-cdex/direct-qu...,-,-,-,-,-,-,[3.2 Benefits](https://hl7.org/fhir/us/davinci...
2,3.3,Sequence Diagram,https://hl7.org/fhir/us/davinci-cdex/direct-qu...,-,-,-,-,-,-,[3.3 Sequence Diagram](https://hl7.org/fhir/us...
3,3.4,Discovery of Patient FHIR IDs,https://hl7.org/fhir/us/davinci-cdex/direct-qu...,-,-,-,-,-,-,[3.4 Discovery of Patient FHIR IDs](https://hl...
4,3.5,Direct Query Transaction Scenarios,https://hl7.org/fhir/us/davinci-cdex/direct-qu...,-,-,-,-,-,-,[3.5 Direct Query Transaction Scenarios](https...
5,3.5.1,Scenario 1,https://hl7.org/fhir/us/davinci-cdex/direct-qu...,-,-,-,-,-,-,[3.5.1 Scenario 1](https://hl7.org/fhir/us/dav...
6,3.5.2,Scenario 2,https://hl7.org/fhir/us/davinci-cdex/direct-qu...,-,-,-,-,-,-,[3.5.2 Scenario 2](https://hl7.org/fhir/us/dav...
7,3.5.3,Scenario 3,https://hl7.org/fhir/us/davinci-cdex/direct-qu...,-,-,-,-,-,-,[3.5.3 Scenario 3](https://hl7.org/fhir/us/dav...
8,3.6,Provenance,https://hl7.org/fhir/us/davinci-cdex/direct-qu...,-,-,-,-,-,-,[3.6 Provenance](https://hl7.org/fhir/us/davin...
9,3.6.1,Example of Direct Query Response Including Pro...,https://hl7.org/fhir/us/davinci-cdex/direct-qu...,-,-,-,-,-,-,[3.6.1 Example of Direct Query Response Includ...


In [39]:
for i,col in enumerate(df.columns):
    print(i,col)

0 IG Section
1 IG Section Title
2 IG Section Link
3 RI Section Title
4 RI Section Link
5 TestScript Item
6 TestScript Item Title
7 TestScript Item Link
8  Comment
9 IG link


In [40]:
for i in df.index:
    try:
      print(i,df.loc[i,'Data Element'])
    except KeyError:
      pass

In [47]:
if in_path.name == 'data-element-mapping.csv':
  def editor_warning(my_file):
    return f'''<!-- {my_file}
  *****************************************************************************************************
  *                                  WARNING: DO NOT EDIT THIS FILE                                   *
  *                                                                                                   *
  * This file is generated by csv_to_markdown_tabler.ipynb. Any edits you make to this file will be   *
  * overwritten                                                                                       *
  * To change the contents of this file, edit input/images/{in_path.name}                     *
  *****************************************************************************************************
  -->\n\n'''

  download_link = f'''The data element mapping table is available as a [CSV](data-element-mapping.csv) and [Excel](data-element-mapping.xlsx) file.'''

  # convert to markdown table 
  # select columns:

  # maps = {#'data_elements_to_submit' : ([0,1,3,4,5,7,8,10,11],[1,4,5,6,8]),
  #         'attachments_to_275' : ([0,1,3,4,5,7,8,10,11],[1,6,4,5,8]),
  #         #'attachments_to_requests_277_278' : ([0,1,3,4,5,7,8,10,11],[5,6,2,3]),
  #         'attachments_to_requests' : ([0,1,3,4,5,7,8,10,11],[1,6,7]),
  #         #'data_elements_to_requests': (None,[1,6,7]),
          
  #         'requests-277_278': (None,[1,7,2,3,9])
  #         }
  # this maps highlights new stuff in columns 2-4, using columns 9-11 as the source
  maps = {
          'attachments_to_275' : ([0,1,2,3,4,5,7,8,10,11],[1,5,4,7]),
          'attachments_to_requests' : ([0,1,2,3,4,5,7,8,10,11],[1,5,6]),
          'q-attachments_to_requests' : ([0,1,2,3,4,5,7,8,10,11,15,16,],[1,5,6]),
          'requests-277_278': (range(0,15),[1,6,2,3,8]),
          'attachments_to_PAS_Bundle' : ([0,1,2,3,4,5,7,8,10,11],[1,9,10]),
          'PAS_Bundle_to_278' : ([0,1,2,3,4,5,7,8,10,11],[1,3,9,10]),
          'PAS_Bundle_to_parameters' : ([0,1,2,3,4,5,7,8,10,11],[1,5,9,10]),
          'PAS_Bundle_to_task' : ([0,1,2,3,4,5,7,8,10,11],[1,6,9,10]),
          }
  # maps = {'text' : ([0,1,3,4,5,7,8,10,11],[1,6,5]) }
  # maps = {'text' : (None,[0,1,2]) }

  # df1 = df[['No','Data Element','CDex $submit-attachment Parameter', 'X12n 275']]
  df1 = df.iloc[:, [0,1,5,4,7]]
else:
   maps = {'text' : (None,[9,4,7
                           
                           
                           
                           
                           
                           
                           
                           
                           
                           
                           
                           
                           
                           ,])}
for k,v in maps.items():
  if k != 'text':

    my_file =f'{k}.md'
    if v[0]:
      df1 = df.iloc[v[0], v[1]]
    else:
      df1 = df.iloc[:, v[1]]
    with open(out_path / my_file, 'w') as md:
      my_table = df1.to_markdown(index=False, tablefmt="github")
      for i in range(1,10):
        my_table = my_table.replace('----','---')
        my_table = my_table.replace('  ',' ')
      print(f'======== my_file: {my_file} =======')
      display(Markdown(my_table))
      md.write(f'{editor_warning(my_file)}{my_table}\n{{:.grid}}\n\n{download_link}')
  else:
    if v[0]:
      df1 = df.iloc[v[0], v[1]]
    else:
      df1 = df.iloc[:, v[1]]
    my_table = df1.to_markdown(index=True, tablefmt="github")
    for i in range(1,10):
      my_table = my_table.replace('----','---')
      my_table = my_table.replace('  ',' ')
    print(my_table)
    display(Markdown(my_table))


| | IG link | RI Section Link | TestScript Item Link |
|---|----------------|---|---|
| 0 | [3 Direct Query](https://hl7.org/fhir/us/davinci-cdex/direct-query.html) | - | - |
| 1 | [3.2 Benefits](https://hl7.org/fhir/us/davinci-cdex/direct-query.html#benefits) | - | - |
| 2 | [3.3 Sequence Diagram](https://hl7.org/fhir/us/davinci-cdex/direct-query.html#sequence-diagram) | - | - |
| 3 | [3.4 Discovery of Patient FHIR IDs](https://hl7.org/fhir/us/davinci-cdex/direct-query.html#discovery-of-patient-fhir-ids) | - | - |
| 4 | [3.5 Direct Query Transaction Scenarios](https://hl7.org/fhir/us/davinci-cdex/direct-query.html#direct-query-transaction-scenarios) | - | - |
| 5 | [3.5.1 Scenario 1](https://hl7.org/fhir/us/davinci-cdex/direct-query.html#scenario-1) | - | - |
| 6 | [3.5.2 Scenario 2](https://hl7.org/fhir/us/davinci-cdex/direct-query.html#scenario-2) | - | - |
| 7 | [3.5.3 Scenario 3](https://hl7.org/fhir/us/davinci-cdex/direct-query.html#scenario-3) | - | - |
| 8 | [3.6 Provenance](ht

| | IG link | RI Section Link | TestScript Item Link |
|---|----------------|---|---|
| 0 | [3 Direct Query](https://hl7.org/fhir/us/davinci-cdex/direct-query.html) | - | - |
| 1 | [3.2 Benefits](https://hl7.org/fhir/us/davinci-cdex/direct-query.html#benefits) | - | - |
| 2 | [3.3 Sequence Diagram](https://hl7.org/fhir/us/davinci-cdex/direct-query.html#sequence-diagram) | - | - |
| 3 | [3.4 Discovery of Patient FHIR IDs](https://hl7.org/fhir/us/davinci-cdex/direct-query.html#discovery-of-patient-fhir-ids) | - | - |
| 4 | [3.5 Direct Query Transaction Scenarios](https://hl7.org/fhir/us/davinci-cdex/direct-query.html#direct-query-transaction-scenarios) | - | - |
| 5 | [3.5.1 Scenario 1](https://hl7.org/fhir/us/davinci-cdex/direct-query.html#scenario-1) | - | - |
| 6 | [3.5.2 Scenario 2](https://hl7.org/fhir/us/davinci-cdex/direct-query.html#scenario-2) | - | - |
| 7 | [3.5.3 Scenario 3](https://hl7.org/fhir/us/davinci-cdex/direct-query.html#scenario-3) | - | - |
| 8 | [3.6 Provenance](https://hl7.org/fhir/us/davinci-cdex/direct-query.html#provenance) | - | - |
| 9 | [3.6.1 Example of Direct Query Response Including Provenance](https://hl7.org/fhir/us/davinci-cdex/direct-query.html#example-of-direct-query-response-including-provenance) | - | - |
| 10 | [3.7 Signatures](https://hl7.org/fhir/us/davinci-cdex/direct-query.html#signatures) | - | - |
| 11 | [3.7.3 Example of Signed Direct Query Response](https://hl7.org/fhir/us/davinci-cdex/direct-query.html#example-of-signed-direct-query-response) | - | - |
| 12 | [4 Task Based Approach](https://hl7.org/fhir/us/davinci-cdex/task-based-approach.html) | - | - |
| 13 | [4.0.5 Discovery of Identifiers](https://hl7.org/fhir/us/davinci-cdex/task-based-approach.html#discovery-of-identifiers) | - | - |
| 14 | [4.0.6 Fetching the Data](https://hl7.org/fhir/us/davinci-cdex/task-based-approach.html#fetching-the-data) | - | - |
| 15 | [4.0.7 Example Transactions Based on Query: Code and Free Text Inputs](https://hl7.org/fhir/us/davinci-cdex/task-based-approach.html#example-transactions-based-on-query-code-and-free-text-inputs) | - | - |
| 16 | [4.0.7.1 Scenario 1](https://hl7.org/fhir/us/davinci-cdex/task-based-approach.html#scenario-1) | - | - |
| 17 | [4.0.7.1.1 FHIR RESTful Query Syntax](https://hl7.org/fhir/us/davinci-cdex/task-based-approach.html#fhir-restful-query-syntax) | - | - |
| 18 | [4.0.7.1.2 Free Text](https://hl7.org/fhir/us/davinci-cdex/task-based-approach.html#free-text) | - | - |
| 19 | [4.0.7.1.3 Contained Task Output](https://hl7.org/fhir/us/davinci-cdex/task-based-approach.html#contained-task-output) | - | - |
| 20 | [4.0.7.2 Scenario 2](https://hl7.org/fhir/us/davinci-cdex/task-based-approach.html#scenario-2) | - | - |
| 21 | [4.0.7.2.1 Progress note Exam Notes as PDF](https://hl7.org/fhir/us/davinci-cdex/task-based-approach.html#progress-note-exam-notes-as-pdf) | - | - |
| 22 | [4.0.7.2.2 Surgical Notes as CCDA Documents](https://hl7.org/fhir/us/davinci-cdex/task-based-approach.html#surgical-notes-as-ccda-documents) | - | - |
| 23 | [4.0.8 Using Questionnaire as Task Input](https://hl7.org/fhir/us/davinci-cdex/task-based-approach.html#using-questionnaire-as-task-input) | - | - |
| 24 | [4.0.8.1 Using Da Vinci DTR to Complete the Questionnaire](https://hl7.org/fhir/us/davinci-cdex/task-based-approach.html#using-da-vinci-dtr-to-complete-the-questionnaire) | - | - |
| 25 | [4.0.8.2 Example of Requesting Data Using A FHIR Questionnaire](https://hl7.org/fhir/us/davinci-cdex/task-based-approach.html#example-of-requesting-data-using-a-fhir-questionnaire) | - | - |
| 26 | [4.0.9 When The Task Cannot Be Completed](https://hl7.org/fhir/us/davinci-cdex/task-based-approach.html#when-the-task-cannot-be-completed) | - | - |
| 27 | [4.0.9.1 Example Unsuccessful Task Based Transaction](https://hl7.org/fhir/us/davinci-cdex/task-based-approach.html#example-unsuccessful-task-based-transaction) | - | - |
| 28 | [4.0.10 Polling vs Subscriptions](https://hl7.org/fhir/us/davinci-cdex/task-based-approach.html#polling-vs-subscriptions) | - | - |
| 29 | [4.0.10.1 Polling](https://hl7.org/fhir/us/davinci-cdex/task-based-approach.html#polling) | - | - |
| 30 | [4.0.10.2 Subscriptions](https://hl7.org/fhir/us/davinci-cdex/task-based-approach.html#subscription) | - | - |
| 31 | [4.0.10.3 Example Task Based Transaction using Subscription](https://hl7.org/fhir/us/davinci-cdex/task-based-approach.html#example-task-based-transaction-using-subscription) | - | - |
| 32 | [4.0.11 Formal Authorization](https://hl7.org/fhir/us/davinci-cdex/task-based-approach.html#formal-authorization) | - | - |
| 33 | [4.0.11.1 Example Task Based Transaction with a Formal Authorization](https://hl7.org/fhir/us/davinci-cdex/task-based-approach.html#example-task-based-transaction-with-a-formal-authorization) | - | - |
| 34 | [4.0.12 Provenance](https://hl7.org/fhir/us/davinci-cdex/task-based-approach.html#provenance) | - | - |
| 35 | [4.0.12.1 Example Requests for Provenance using Task Based Transaction](https://hl7.org/fhir/us/davinci-cdex/task-based-approach.html#example-requests-for-provenance-using-task-based-transaction) | - | - |
| 36 | [4.0.13 Signatures](https://hl7.org/fhir/us/davinci-cdex/task-based-approach.html#signatures) | - | - |
| 37 | [4.0.13.3 Example of a Signed Task Based Transaction](https://hl7.org/fhir/us/davinci-cdex/task-based-approach.htmtask-based-approach.html#example-of-a-signed-task-based-transactionl#signatures) | - | - |
| 38 | [4.1 Conforming to CDex Task Based Approach](https://hl7.org/fhir/us/davinci-cdex/task-based-conformance.html) | - | - |
| 39 | [5.1 Solicited and Unsolicited Attachments](https://hl7.org/fhir/us/davinci-cdex/solicited-unsolicited-attachments.html) | - | - |
| 40 | [5.2 Sending Attachments](https://hl7.org/fhir/us/davinci-cdex/sending-attachments.html) | - | - |
| 41 | [5.3 Requesting Attachments Using Attachments Codes](https://hl7.org/fhir/us/davinci-cdex/requesting-attachments-code.html) | - | - |
| 42 | [5.4 Requesting Attachments Using Questionnaires](https://hl7.org/fhir/us/davinci-cdex/requesting-attachments-questionnaire.html) | - | - |
| 43 | [5.5 Using CDex Attachments with DaVinci PAS](https://hl7.org/fhir/us/davinci-cdex/burden-reduction.html) | - | - |
| 44 | [5.6 Conforming to CDex Attachments](https://hl7.org/fhir/us/davinci-cdex/attachments-conformance.html) | - | - |
| 45 | [6 About Signatures](https://hl7.org/fhir/us/davinci-cdex/signatures.html) | - | - |
| 46 | [6.0.5 Electronic Signatures](https://hl7.org/fhir/us/davinci-cdex/signatures.html#electronic-signatures) | - | - |
| 47 | [6.0.6 Digital Signatures](https://hl7.org/fhir/us/davinci-cdex/signatures.html#digital-signatures) | - | - |
| 48 | [7 Security and Privacy](https://hl7.org/fhir/us/davinci-cdex/security.html) | - | - |